In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import bokeh as bk
from collections import Counter


In [2]:
### Import data

data_dir = '/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs'
filename = 'Vahid_std.csv'
filepath = os.path.join(data_dir,filename)

df = pd.read_csv(filepath)
#print(df)

In [3]:
hyst_col = {'n_cycles', 'hysteresis_area', 'stiffness'}
hyst_df = pd.DataFrame(columns = hyst_col)
#hyst_df.n_cycles = n_cycles


In [4]:
### Extract number of cycles without repeating values in other table, and store number of measurements per cycle

n_cycles = sorted(set(df.Machine_N_cycles))
n_measurements = Counter(df.Machine_N_cycles)  


#print(n_cycles)
#print(n_measurements)
n_cycles_df = pd.DataFrame(n_cycles)
#n_cycles_df.to_csv(path_or_buf='/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs/Vahid_n_cycles.csv', index=False)

hyst_df.n_cycles = n_cycles



In [5]:
### Isolate Stress/Strain values for each cycles


Stress_N = []
Strain_N = []
df0=np.array(df)
for k in range(len(n_cycles)):
    Stress_N.append([df0[i,14] for i in range(df0.shape[0]) if df0[i,1]==n_cycles[k]])
    Strain_N.append([df0[i,0] for i in range(df0.shape[0]) if df0[i,1]==n_cycles[k]])
    
    
#print(Stress_N)
#print(Strain_N)
#print(len(Stress_N))

In [6]:
### Definition of polyarea and stiffness
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))


In [7]:
### Hysteresis loops area computation

Hysteresis_Area = []
Stiffness = []

for j in range(len(n_cycles)):
    x=Stress_N[j]
    y=Strain_N[j]
    points=(x, y)
    Hysteresis_Area.append(PolyArea(x, y))
    #Stress_min = np.min


hyst_df.hysteresis_area = Hysteresis_Area
#hyst_df.to_csv(path_or_buf='/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs/Vahid_hystarea.csv', index=False)



In [8]:
### Stiffness computation


Stress_max = [np.max(Stress_N[i]) for i in range(1,len(n_cycles))]
Stress_min = [np.min(Stress_N[i]) for i in range(1,len(n_cycles))]
Strain_max = [np.max(Strain_N[i]) for i in range(1,len(n_cycles))]
Strain_min = [np.min(Strain_N[i]) for i in range(1,len(n_cycles))]
                    
Stiffness = []

for i in range(len(n_cycles)-1):
    dStress = Stress_max[i]-Stress_min[i]
    dStrain = Strain_max[i]-Strain_min[i]
    Stiffness.append(dStress/dStrain)

Stiffness.append(1)    
hyst_df.stiffness = Stiffness
#stif_df.to_csv(path_or_buf='/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs/Vahid_stiff.csv', index=False)

In [9]:
print(hyst_df)

      hysteresis_area     stiffness   n_cycles
0            0.000000  43687.822473        NaN
1            0.000401  29217.481622        1.0
2            0.001224  27391.381050     1000.0
3            0.001222  25264.809740     2000.0
4            0.003683  23663.124346     3000.0
...               ...           ...        ...
1194         0.032850   6937.089743  1193000.0
1195         0.035368   6698.079021  1194000.0
1196         0.033975   6566.353163  1195000.0
1197         0.033212   6331.946706  1196000.0
1198         0.032153      1.000000  1197000.0

[1199 rows x 3 columns]


In [10]:
hyst_df.to_csv(path_or_buf='/Users/scottmatthewssalmon/Desktop/github/CCFatigue/Outputs/Vahid_hysteresis_analysis.csv', index=False)
